In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv('/kaggle/input/disease-prediction-using-machine-learning/Training.csv')
test = pd.read_csv('../input/disease-prediction-using-machine-learning/Testing.csv')
data = train.copy()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.drop('Unnamed: 133', axis=1, inplace=True)
data['prognosis'].nunique()

In [ ]:
data['prognosis'].value_counts()

In [ ]:
x = data.iloc[:, :-1]

In [ ]:
#def select_symtom(disease):
 #   symtoms = set()
  #  for col in x.columns:
        

## Finding Diseases from Symptoms.

**Lets see that in which which diseases the particular symptom occur**

In [ ]:
symtoms = data.groupby('prognosis')['itching'].value_counts()
symtoms = symtoms.to_frame()
symtoms.head()

In [ ]:
smpt = []
l = list(symtoms[symtoms['itching'] < 120].index)
for i,j in l:
    if j == 1:
        smpt.append(i)

In [ ]:
smpt

In [ ]:
l= data[data['prognosis']=="Fungal infection"]['itching'].value_counts().to_frame()
print(l)

In [ ]:
sm=[]
if l['itching'][1] < 120:
    sm.append('itching')
            
print(sm)

### Finding Symptoms list from Disease

**Now we will get the list of symptoms that can be seen in particular disease**

In [ ]:
def get_symptoms(disease):
    sympt = []
    for col in x.columns:
        lst = data[data['prognosis']==disease][col].value_counts().to_frame()
        try:
            if lst[col][1] < 120:
                sympt.append(col)
        except:
            continue
    return sympt

In [ ]:
get_symptoms("Fungal infection")

In [ ]:
def get_symptoms2(disease):
    sympt = []
    for col in x.columns:
        lst = data[data['prognosis']==disease][col].value_counts().to_frame()
        if(len(lst.index) > 1):
            sympt.append(col)
    return sympt

In [ ]:
get_symptoms2("Fungal infection")

In [ ]:
#data['Symptoms'] = data['prognosis'].apply(lambda x: get_symptoms2(x))

In [ ]:
sympt_col = x.columns
disease = data['prognosis'].unique()
#disease

In [ ]:
sympt_dict = {}
for dis in disease:
    all_sympt = get_symptoms2(dis)
    sympt_dict[dis] = all_sympt
    
#print(sympt_dict)
for key, value in sympt_dict.items():
    print("Disease: ", key)
    print("Symptoms: ", value)

**This Dictionary is very much helpful for us. We will be usig this dictionary while deploying.**

And we can also map this values and create a new column as symptom infront of disease and can has a wonderful analysis of all the symptoms.

In [ ]:
import pickle
file = open("diseas_sympt.pkl","wb")
pickle.dump(sympt_dict, file)

In [ ]:
data['Symptoms'] = data['prognosis'].map(sympt_dict)

In [ ]:
data.tail()

**Categorical Encoding of Prognosis using Integer label encoding**

In [ ]:
#disease = data['prognosis'].unique()
cat_encode = {k: i for i, k in enumerate(disease, 0)}
data['prognosis'] = data['prognosis'].map(cat_encode)

## Let's try modelling the prognosis once

In [ ]:
X = data[x.columns]
Y = data['prognosis']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=11)

In [ ]:
print(x_train.shape)
print(x_test.shape)
#check all the class data we have in training or not
#len(set(y_train))
#len(set(y_test))

In [ ]:
dt_model = DecisionTreeClassifier()
dt_model.fit(x_train, y_train)

In [ ]:
y_pred = dt_model.predict(x_test)
#print(y_pred)

In [ ]:
print(accuracy_score(y_test, y_pred))

## Encoding Test set
we are having prognosis column in test set as well.

In [ ]:
test.head(2)

In [ ]:
test['prognosis'] = test['prognosis'].map(cat_encode)

In [ ]:
x_testing = test[x.columns]
y_testing = test['prognosis']

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X, Y)

In [ ]:
predictions = dt.predict(x_testing)
print("accuracy: ", accuracy_score(y_testing, predictions))

* This is very good that decisionTree gives the overall accuracy of 97%.
* Let's try some different models too

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X,Y)
rf_pred = rf.predict(x_testing)
print("accuracy: ", accuracy_score(y_testing, rf_pred))

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X,Y)
nb_pred = nb.predict(x_testing)
print("accuracy: ", accuracy_score(y_testing, rf_pred))

## Save the Model

In [ ]:
import pickle
f = open("disease_predDT.pkl", "wb")
pickle.dump(dt, f)

In [ ]:
model = pickle.load(open('disease_predDT.pkl', "rb"))
#model = pickle.load(open('doctorrf_fees.pkl','rb'))

In [ ]:
predi = model.predict(x_testing)
print("accuracy: ", accuracy_score(y_testing, predi))

In [ ]:
#pickle.dump(sympt_dict,open("diseases.pkl","wb"))

In [ ]:
#dic = pickle.load(open("diseases.pkl","rb"))

**How we will display the name of disease predicted**


In [ ]:
key_list = list(cat_encode.keys())
val_list = list(cat_encode.values())

In [ ]:
p = 27
posi = val_list.index(27)
print(key_list[posi])

In [ ]:
#so let's save the encoding disease dictionary too
pickle.dump(cat_encode, open("diseases.pkl","wb"))

In [ ]:
dis = pickle.load(open("diseases.pkl","rb"))

In [ ]:
pickle.dump(sympt_dict, open("diseaseWithSymp.pkl","wb"))

In [ ]:
dissym = pickle.load(open("diseaseWithSymp.pkl","rb"))

In [ ]:
symptoms = x.columns

In [ ]:
pickle.dump(symptoms, open("symptoms.pkl","wb"))

In [ ]:
spt = pickle.load(open("symptoms.pkl","rb"))

In [ ]:
#how to change all the values to 0 except the user enter symptom as 1

In [ ]:
smptd={}
for sympt in spt:
    if sympt == "watering_from_eyes" or sympt=="shivering" or sympt=="chills" or sympt=="vomiting":
        smptd[sympt] = 1
    else:
        smptd[sympt] = 0

In [ ]:
moddf = pd.DataFrame(smptd, columns=smptd.keys(), index=smptd.values())

In [ ]:
t = np.array(moddf.iloc[0])

In [ ]:
output = model.predict(moddf)
output

In [ ]:
pos = val_list.index(output)
print(key_list[6])

In [ ]:
sympt_dict

In [ ]:
t